# PairCorr Side-Band Benchmark
Synthetic test: injects a 7% side-band at $k_1 = k_0(1+1/14)$ and runs the TORUS detection pipeline.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy.signal import periodogram
from scipy.signal.windows import hann

# Parameters
N = 1000
L = 100
k0 = 2 * np.pi / (L / 10)
k1 = k0 * (1 + 1/14)
amp0 = 1.0
amp1 = 0.07

# Generate synthetic (x, y) snapshot
theta = np.random.uniform(0, 2*np.pi, N)
r = L/2 + (L/4) * np.sin(k0 * np.arange(N) + np.random.uniform(0,2*np.pi))
x = r * np.cos(theta)
y = r * np.sin(theta)

# Inject side-band
r_sb = L/2 + (L/4) * (amp0 * np.sin(k0 * np.arange(N)) + amp1 * np.sin(k1 * np.arange(N)))
x_sb = r_sb * np.cos(theta)
y_sb = r_sb * np.sin(theta)

# Compute pairwise distances
from scipy.spatial.distance import pdist
dists = pdist(np.column_stack([x_sb, y_sb]))

# Bin to g2(r)
bins = np.linspace(0, L, 200)
g2, _ = np.histogram(dists, bins=bins)
g2 = g2 / g2.max()
r_centers = 0.5 * (bins[1:] + bins[:-1])

# FFT
windowed = g2 * hann(len(g2))
freqs, power = periodogram(windowed, scaling='spectrum')
k0_idx = np.argmax(power)
k0_val = freqs[k0_idx]
k1_val = k0_val * (1 + 1/14)
k1_idx = np.argmin(abs(freqs - k1_val))
deltaP = 10 * np.log10(power[k1_idx] / power[k0_idx])
plt.plot(freqs, 10*np.log10(power)); plt.axvline(k0_val, color='b'); plt.axvline(k1_val, color='r');
plt.title('Power Spectrum'); plt.xlabel('k'); plt.ylabel('10log10(Power)'); plt.show()
print('k0=', k0_val, 'k1=', k1_val, 'ΔP=', deltaP, 'dB')
if abs(k1_val-freqs[k1_idx])/k1_val<=.03 and -45<deltaP<-25:
    print('TORUS-POSITIVE')
else:
    print('TORUS-NEGATIVE')